# 3_GitlabRunner_ShellTest – Running GitLab CI Jobs Locally

This notebook demonstrates how to use `gitlab-runner` with the `shell` executor to test CI jobs directly on your local system. It enables fast, local validation of `.gitlab-ci.yml` configurations without requiring Docker or pushing to a remote GitLab server.

In [5]:
import subprocess
import os
from pathlib import Path

def in_jupyter():
    try:
        get_ipython
        return True
    except NameError:
        return False

home = Path.home()
proj = home / "gitlab_demo_shell"
env = os.environ.copy()
env["PATH"] = f"{home}/bin:" + env["PATH"]
env["SHELL"] = "/bin/bash"

proj.mkdir(parents=True, exist_ok=True)

(proj / "test_sample.py").write_text("""\
def add(a, b):
    return a + b

def test_add():
    assert add(2, 2) == 4
""")

(proj / ".gitlab-ci.yml").write_text("""\
stages:
  - test

pytest:
  stage: test
  script:
    - pytest
""")

if not (proj / ".git").exists():
    subprocess.run("git init", shell=True, cwd=proj, env=env, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run("git config user.name 'CI Tester'", shell=True, cwd=proj, env=env)
    subprocess.run("git config user.email 'ci@test.local'", shell=True, cwd=proj, env=env)
    subprocess.run("git add .", shell=True, cwd=proj, env=env)
    subprocess.run("git commit -m 'initial commit'", shell=True, cwd=proj, env=env, stdout=subprocess.DEVNULL)

if in_jupyter():
    print("⚠️ Warning: This notebook is running inside Jupyter.")
    print("   GitLab Runner with the 'shell' executor may fail due to missing login shell features.\n")
    print("🔁 If this fails, open a terminal and run manually:")
    print(f"   cd {proj}")
    print("   gitlab-runner exec shell pytest\n")

print(f"\n🚀 Running GitLab CI job using shell executor in: {proj}\n")

try:
    subprocess.run(
        "gitlab-runner exec shell pytest",
        shell=True,
        cwd=proj,
        env=env,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        check=True
    )
    print("✅ GitLab CI job ran successfully (shell executor).")
except subprocess.CalledProcessError:
    print("❌ GitLab CI job failed.")
    print("   Please try again in a regular terminal if this happened inside Jupyter.")


⚠️ Warning: This notebook is running inside Jupyter.
   GitLab Runner with the 'shell' executor may fail due to missing login shell features.

🔁 If this fails, open a terminal and run manually:
   cd /home/rolan/gitlab_demo_shell
   gitlab-runner exec shell pytest


🚀 Running GitLab CI job using shell executor in: /home/rolan/gitlab_demo_shell

❌ GitLab CI job failed.
   Please try again in a regular terminal if this happened inside Jupyter.
